In [ ]:
import shutil
import hyp3_sdk as sdk
from asf_tools.water_map import make_water_map

hyp3 = sdk.HyP3(prompt=True)

In [ ]:
granule = 'S1A_IW_GRDH_1SDV_20210413T235641_20210413T235706_037439_0469D0_3F2B'
job = hyp3.submit_rt_job(granule, scale='power')

In [ ]:
job = hyp3.watch(job)  # will take ~25 minutes to finish processing the On Demand RTC job
product_zip = job.download_files()[0]

In [ ]:
shutil.unpack_archive(product_zip)
vv_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VV.tif'
vh_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VH.tif'

In [ ]:
from asf_tools.composite import get_epsg_code
from osgeo import gdal

vh_info = gdal.Info(vh_raster, format='json')
vh_epsg = get_epsg_code(vh_info)
vh_bounds = [*vh_info['cornerCoordinates']['upperLeft'], *vh_info['cornerCoordinates']['lowerRight']]

hand_source = '/vsicurl/https://hyp3-testing.s3-us-west-2.amazonaws.com/asf-tools/water-map/mississippi/' \
              'mississippi_10m_hand_cop-glo-30.tif'
hand_raster = vh_raster.replace('_VH.tif', '_hand.tif')
gdal.Warp(hand_raster, hand_source, dstSRS=f'EPSG:{vh_epsg}', 
          outputBounds=vh_bounds, width=vh_info['size'][0], height=vh_info['size'][1])

In [ ]:
help(make_water_map)

In [ ]:
water_extent_raster = 'water_extent.tif'
make_water_map(water_extent_raster, vv_raster, vh_raster, hand_raster, tile_shape=(100, 100), 
               max_vv_threshold=-17., max_vh_threshold=-24., hand_threshold=15., hand_fraction=0.8)